In [3]:
%pip install gradientai --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

os.environ['GRADIENT_ACCESS_TOKEN'] = "xxx"
os.environ['GRADIENT_WORKSPACE_ID'] = "xxx"

In [10]:
from gradientai import Gradient

def main():
  with Gradient() as gradient:
      base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

      new_model_adapter = base_model.create_model_adapter(
          name="test model 3"
      )
      print(f"Created model adapter with id {new_model_adapter.id}")
      sample_query = "### Instruction: Write the Python Code for the following exercise : Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets. Function definition : def threeSum(nums: List[int]) -> List[List[int]]: \n\n### Response:"
      print(f"Asking: {sample_query}")

      # before fine-tuning
      completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=511).generated_output
      print(f"Generated (before fine-tune): {completion}")

if __name__ == "__main__":
    main()

Created model adapter with id 4c764741-1eb5-4ff1-ab62-388df1027eb8_model_adapter
Asking: ### Instruction: Write the Python Code for the following exercise : Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets. Function definition : def threeSum(nums: List[int]) -> List[List[int]]: 

### Response:
Generated (before fine-tune): 
 Here's the Python code for the given exercise:
```python
def threeSum(nums: List[int]) -> List[List[int]]:
    result = []
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            for k in range(j+1, len(nums)):
                if nums[i] + nums[j] + nums[k] == 0 and i != j and i != k and j != k:
                    result.append([nums[i], nums[j], nums[k]])
    return result
```
This function takes in a list of integers as input and returns a list of lists containing t

After this first test, I ran the code on leetcode and got the following result: Wrong Answer 45 / 312 testcases passed. Disappointing result, what's more the code isn't top-notch in terms of time complexity - we could do a lot better. In the next step, I'll try to fine-tune the ia and give it a better prompt in order to get a result that works.

In [15]:
from gradientai import Gradient

def main():
  with Gradient() as gradient:
      base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

      new_model_adapter = base_model.create_model_adapter(
          name="test model 3"
      )
      print(f"Created model adapter with id {new_model_adapter.id}")
      sample_query = "### Instruction: Write an optimal solution in terms of space and time complexity for the following exercise : Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0, Notice that the solution set must not contain duplicate triplets. Function definition : def threeSum(nums: List[int]) -> List[List[int]]: Write the full python code. \n\n### Response:"
      print(f"Asking: {sample_query}")

      samples = [
        {
            "inputs": "### Instruction: Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets. \n\n### Response:",
            "response": "class Solution:\n    def threeSum(self, nums: List[int]) -> List[List[int]]:\n        res = set()\n        n, p, z = [], [], []\n        for num in nums:\n            if num > 0:\n                p.append(num)\n            elif num < 0:\n                n.append(num)\n            else:\n                z.append(num)\n        N, P = set(n), set(p)\n        if z:\n            for num in P:\n                if -1*num in N:\n                    res.add((-1*num, 0, num))\n        if len(z) >= 3:\n            res.add((0,0,0))\n        for i in range(len(n)):\n            for j in range(i+1,len(n)):\n                target = -1*(n[i]+n[j])\n                if target in P:\n                    res.add(tuple(sorted([n[i],n[j],target])))\n        for i in range(len(p)):\n            for j in range(i+1,len(p)):\n                target = -1*(p[i]+p[j])\n                if target in N:\n                    res.add(tuple(sorted([p[i],p[j],target])))\n        return res\n"
        },
        {
            "inputs": "### Instruction: Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets. \n\n### Response:",
            "response": "class Solution:\n    def threeSum(self, nums: List[int]) -> List[List[int]]:\n        nums.sort()\n        triplets = set()\n        for i in range(len(nums) - 2):\n            firstNum = nums[i]\n            j = i + 1\n            k = len(nums) - 1\n            while j < k:\n                secondNum  = nums[j]\n                thirdNum = nums[k]\n                potentialSum = firstNum + secondNum + thirdNum\n                if potentialSum > 0:\n                    k -= 1\n                elif potentialSum < 0:\n                    j += 1\n                else:\n                    triplets.add((firstNum , secondNum ,thirdNum))\n                    j += 1\n                    k -= 1\n        return triplets\n"
        },
        {
            "inputs": "### Instruction: Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0. Notice that the solution set must not contain duplicate triplets. \n\n### Response:",
            "response": "class Solution:\n    def threeSum(self, nums: List[int]) -> List[List[int]]:\n        nums.sort()\n        l=[]\n        for i in range(len(nums)):\n            if i>0 and nums[i-1]==nums[i]:\n                continue\n            j=i+1\n            k=len(nums)-1\n            while j<k:\n                s=nums[i]+nums[j]+nums[k]\n                if s>0:\n                    k-=1\n                elif s<0:\n                    j+=1\n                else:\n                    l.append([nums[i],nums[j],nums[k]])\n                    j+=1\n                    while nums[j-1]==nums[j] and j<k:\n                        j+=1\n        return l\n"
        }
    ]


      # this is where fine-tuning happens
      # num_epochs is the number of times you fine-tune the model
      # more epochs tends to get better results, but you also run the risk of "overfitting"
      # play around with this number to find what works best for you
      num_epochs = 3
      count = 0
      while count < num_epochs:
          print(f"Fine-tuning the model, iteration {count + 1}")
          new_model_adapter.fine_tune(samples=samples)
          count = count + 1

      # after fine-tuning
      completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=511).generated_output
      print(f"Generated (after fine-tune): {completion}")

      new_model_adapter.delete()

if __name__ == "__main__":
    main()

Created model adapter with id 35818662-e44c-4434-9cd6-b6c0d975d4ce_model_adapter
Asking: ### Instruction: Write an optimal solution in terms of space and time complexity for the following exercise : Given an integer array nums, return all the triplets [nums[i], nums[j], nums[k]] such that i != j, i != k, and j != k, and nums[i] + nums[j] + nums[k] == 0, Notice that the solution set must not contain duplicate triplets. Function definition : def threeSum(nums: List[int]) -> List[List[int]]: Write the full python code. 

### Response:
Fine-tuning the model, iteration 1
Fine-tuning the model, iteration 2
Fine-tuning the model, iteration 3
Generated (after fine-tune): 
 Here's the Python code to solve the problem:
```
def threeSum(nums: List[int]) -> List[List[int]]:
    result = []
    nums.sort()
    for i in range(len(nums)):
        for j in range(i+1, len(nums)):
            for k in range(j+1, len(nums)):
                if nums[i] + nums[j] + nums[k] == 0:
                    result.

After a few prompt tests (having had responses that contained no code, or abbreviated code, I finally got a complete solution), I finally get a result. Unfortunately, the solution doesn't work.
With this first test, I realized that there were several difficulties in my project. The first is data recovery and formatting! Each LLM must receive data with a certain formatting to be fine tuned. What's more, as my answers are code and not sentences, the code itself has to be formatted so that the indentations are correct. This doesn't make the task any easier :o